# Genre Classification EDA
1. Get *n* number of tracks for each genre in genres list via Spotify API
2. Get features of each track via Spotify API
3. Explore data

## Get Access Token

In [1]:
import os
import json
import requests
import pandas as pd

In [43]:
# get query parameters for access token
# make sure you restart the kernel
# if you just have created the environment variables
grant_type = 'client_credentials'
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

In [45]:
# get access token
# it will be used as header in API calls
access_token_url = 'https://accounts.spotify.com/api/token?grant_type={}&client_id={}&client_secret={}'.format(grant_type, client_id, client_secret)
response = requests.post(access_token_url, headers={'Content-Type':'application/x-www-form-urlencoded'})
access_token = 'Bearer ' + json.loads(response.text)['access_token']

print('Access Token:', access_token)

Access Token: Bearer BQDI0K0V9AgXZNgDLL4FNUSGNIe4IHGIr6qE-cl3sZ6IulQbBjvbrR0GdzCkoC5E-t3fN5zUkMjmOcd-KO8X0_pv9EYdrNc_-t9KfXMnVIUh9fnvY15I


## Get Tracks
|track_id|track_name|artist_name|popularity|genre|
|---|---|---|---|---|

In [4]:
# list of genres to be classified
genres_list = ['rap', 'rock', 'metal', 'blues', 'jazz', 'classical', 'funk', 'techno', 'electronic', 'r&b']

In [5]:
steps = 2
limit = 50 # max 50 allowed
offset = 0 # takes the n-1th value of limit

In [6]:
# get tracks for each genre
# TODO: get full list of artists if there are multiple artists present

tracks_df = pd.DataFrame(columns=['track_id', 'track_name', 'artist_name', 'popularity', 'genre'])

for genre in genres_list:

    for step in range(steps):
        search = requests.get('https://api.spotify.com/v1/search?q=genre:{}&type=track&limit={}&offset={}'.format(genre, limit, offset), headers={'Authorization':access_token})
        search_result = json.loads(search.text)

        for n in range(limit):
            track_id = search_result['tracks']['items'][n]['id']
            track_name = search_result['tracks']['items'][n]['name']
            artist_name = search_result['tracks']['items'][n]['artists'][0]['name']
            popularity = search_result['tracks']['items'][n]['popularity']

            tracks_df = tracks_df.append({
                'track_id': track_id,
                'track_name': track_name,
                'artist_name': artist_name,
                'popularity': popularity,
                'genre': genre
            }, ignore_index=True)

        offset += limit

In [7]:
len(tracks_df)

1000

In [8]:
tracks_df

,track_id,track_name,artist_name,popularity,genre
0,4FyesJzVpA39hbYvcseO2d,Just Wanna Rock,Lil Uzi Vert,90,rap
1,7aRCf5cLOFN1U7kvtChY1G,Search & Rescue,Drake,91,rap
2,1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,94,rap
3,2dHHgzDwk4BJdRwy9uXhTO,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,96,rap
4,7KA4W4McWYRpgf0fWsJZWB,See You Again (feat. Kali Uchis),"Tyler, The Creator",92,rap
...,...,...,...,...,...
995,3Wrjm47oTz2sjIgck11l5e,Beggin',Måneskin,85,r&b
996,4gMgiXfqyzZLMhsksGmbQV,"Another Brick in the Wall, Pt. 2",Pink Floyd,78,r&b
997,1ThmUihH9dF8EV08ku5AXN,Faucet Failure,Ski Mask The Slump God,75,r&b
998,0EgLxY52mpGsXETyEsgVlP,HOPE,NF,77,r&b


## Get Track Features

In [26]:
track_features_url = 'https://api.spotify.com/v1/audio-features/'
track_features_df = pd.DataFrame()

for index, row in tracks_df.iterrows():
    track_id = tracks_df.iloc[index]['track_id']
    request_url = track_features_url + track_id
    track_features = requests.get(request_url, headers={'Authorization': access_token})
    request_result = json.loads(track_features.text)
    track_features_df = track_features_df.append(request_result, ignore_index=True)

# drop negligible features
track_features_df.drop(columns=['type', 'uri', 'track_href', 'analysis_url'], inplace=True)
track_features_df.rename(columns={'id':'track_id'}, inplace=True)

In [37]:
# join tracks & features
df = tracks_df.merge(track_features_df, on='track_id')

In [38]:
df

,track_id,track_name,artist_name,popularity,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,4FyesJzVpA39hbYvcseO2d,Just Wanna Rock,Lil Uzi Vert,90,rap,0.486,0.545,11.0,-7.924,1.0,0.0336,0.0652,0.004740,0.0642,0.0385,150.187,123891.0,4.0
1,7aRCf5cLOFN1U7kvtChY1G,Search & Rescue,Drake,91,rap,0.817,0.440,10.0,-8.482,0.0,0.0734,0.0603,0.000001,0.3300,0.5440,142.024,272113.0,4.0
2,1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,94,rap,0.644,0.735,8.0,-5.747,1.0,0.0391,0.0521,0.144000,0.1610,0.4180,88.980,153947.0,4.0
3,2dHHgzDwk4BJdRwy9uXhTO,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,96,rap,0.715,0.620,1.0,-6.005,0.0,0.0484,0.4170,0.000000,0.0822,0.1720,97.950,221520.0,4.0
4,7KA4W4McWYRpgf0fWsJZWB,See You Again (feat. Kali Uchis),"Tyler, The Creator",92,rap,0.558,0.559,6.0,-9.222,1.0,0.0959,0.3710,0.000007,0.1090,0.6200,78.558,180387.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,3Wrjm47oTz2sjIgck11l5e,Beggin',Måneskin,85,r&b,0.714,0.800,11.0,-4.808,0.0,0.0504,0.1270,0.000000,0.3590,0.5890,134.002,211560.0,4.0
1002,4gMgiXfqyzZLMhsksGmbQV,"Another Brick in the Wall, Pt. 2",Pink Floyd,78,r&b,0.693,0.394,0.0,-15.882,1.0,0.0428,0.0782,0.000694,0.2470,0.7210,104.114,238747.0,4.0
1003,1ThmUihH9dF8EV08ku5AXN,Faucet Failure,Ski Mask The Slump God,75,r&b,0.935,0.552,10.0,-9.373,0.0,0.3350,0.1110,0.000000,0.0952,0.6150,99.993,145627.0,4.0
1004,0EgLxY52mpGsXETyEsgVlP,HOPE,NF,77,r&b,0.546,0.787,0.0,-4.023,0.0,0.2120,0.3960,0.000000,0.0619,0.4970,100.559,264473.0,3.0


In [46]:
broken = requests.get("https://api.spotify.com/v1/tracks/5GorCbAP4aL0EJ16frG2hd", headers={'Authorization': access_token})
json.loads(broken.text)

{'album': {'album_group': 'compilation',
  'album_type': 'compilation',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7oPftvlwr6VrsViSDV7fJY'},
    'href': 'https://api.spotify.com/v1/artists/7oPftvlwr6VrsViSDV7fJY',
    'id': '7oPftvlwr6VrsViSDV7fJY',
    'name': 'Green Day',
    'type': 'artist',
    'uri': 'spotify:artist:7oPftvlwr6VrsViSDV7fJY'}],
  'available_markets': ['AD',
   'AG',
   'AL',
   'AM',
   'AO',
   'AR',
   'AT',
   'AU',
   'AZ',
   'BA',
   'BB',
   'BD',
   'BE',
   'BG',
   'BI',
   'BJ',
   'BO',
   'BR',
   'BS',
   'BT',
   'BW',
   'BZ',
   'CA',
   'CD',
   'CG',
   'CH',
   'CI',
   'CL',
   'CM',
   'CO',
   'CR',
   'CV',
   'CW',
   'CY',
   'CZ',
   'DE',
   'DJ',
   'DK',
   'DM',
   'DO',
   'DZ',
   'EC',
   'EE',
   'ES',
   'ET',
   'FI',
   'FJ',
   'FM',
   'FR',
   'GA',
   'GB',
   'GD',
   'GE',
   'GH',
   'GM',
   'GN',
   'GQ',
   'GR',
   'GT',
   'GW',
   'GY',
   'HK',
   'HN',
   'HR',
   'HT',
   'HU',
